In [2]:
import nltk
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to C:\Users\Bisol
[nltk_data]     Mathai\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd

In [4]:
# Load the dataset
df = pd.read_csv('processed_dataset_20000.csv')
print(df.columns)

Index(['user_name', 'user_location', 'user_description', 'user_created',
       'user_followers', 'user_friends', 'user_favourites', 'user_verified',
       'date', 'text', 'hashtags', 'source', 'is_retweet',
       'friends_follower_ratio', 'Bot Label', 'cleaned_text', 'hashtag_count',
       'mention_count', 'url_count'],
      dtype='object')


In [5]:
# Initialize the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()
def get_sentiment_score(text):
    if isinstance(text, str):
        return analyzer.polarity_scores(text)['compound']
    else:
        return 0.0  # Neutral score for non-string entries

# Function to assign sentiment labels based on compound score
def get_sentiment_label(score):
    if score >= 0.05:
        return 'Positive'
    elif score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# Apply sentiment analysis to the 'cleaned_text' column
df['sentiment_score'] = df['cleaned_text'].apply(get_sentiment_score)

# Apply sentiment labeling
df['sentiment'] = df['sentiment_score'].apply(get_sentiment_label)

# Save the updated DataFrame to a new CSV file
# df.to_csv('labeled_sentiment_1.csv', index=False)

In [7]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Load dataset
df = pd.read_csv('labeled_sentiment.csv')

# Fix: Remove or fill NaNs
# df['cleaned_text'] = df['cleaned_text'].fillna('')  # or use dropna(subset=['cleaned_text'])
df = df.dropna(subset=['cleaned_text'])

# Initialize vectorizer
vectorizer = CountVectorizer()

# Fit and transform
X_bow = vectorizer.fit_transform(df['cleaned_text'])

# Convert to DataFrame
bow_df = pd.DataFrame(X_bow.toarray(), columns=vectorizer.get_feature_names_out())

# Show result
print(bow_df)


      ___  _______  _______________________________________  _____are  ____i  \
0       0        0                                        0         0      0   
1       0        0                                        0         0      0   
2       0        0                                        0         0      0   
3       0        0                                        0         0      0   
4       0        0                                        0         0      0   
...   ...      ...                                      ...       ...    ...   
9910    0        0                                        0         0      0   
9911    0        0                                        0         0      0   
9912    0        0                                        0         0      0   
9913    0        0                                        0         0      0   
9914    0        0                                        0         0      0   

      _ツ_  aaveusd  aback  abandon  abb

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Define features and target variable
X = X_bow
y = df['sentiment']  # Ensure this column contains the sentiment labels

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the SVM classifier
svm_clf = SVC(kernel='linear')
svm_clf.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_clf.predict(X_test)

# Evaluate the classifier
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

    Negative       0.82      0.76      0.79       270
     Neutral       0.93      0.97      0.95       960
    Positive       0.94      0.91      0.92       753

    accuracy                           0.92      1983
   macro avg       0.90      0.88      0.89      1983
weighted avg       0.92      0.92      0.92      1983

